# Part 06 - Molecular dynamics example

This is a toy "spring" particles trapped
in a spring well, using a symplectic
integrator. Great example of python's
power (and its lack of speed). Balance
<em>your</em> programming time against the <em>run
time</em> when choosing scripting or compiled
languages.

In [1]:
import numpy, math
box_length = 10
N = 20
dt = 0.1
k_particle = 5
diameter = 2.0
k_domain = 0.1
pos = (numpy.random.rand(N, 2) - 0.5) * box_length
vel = numpy.zeros((N,2))

def force(i, j):
    rij = pos[i] - pos[j]
    distance = numpy.linalg.norm(rij)
    if distance > diameter or i == j:
        return numpy.array([0,0])
    rij_norm = rij / distance
    return k_particle * (diameter - distance) * rij_norm

from ipywidgets import FloatProgress
from IPython.display import display
f = FloatProgress(min=0, max=100)
display(f)

def timestep(step):
    for i in range(N):
        pos[i] += 0.5 * dt * vel[i]

    forces = numpy.zeros((N,2))
    for i in range(N):
        forces[i] += - k_domain * pos[i]
        for j in range(N):
            forces[i] += force(i,j)

    for i in range(N):
        vel[i] += dt * forces[i]
    
    for i in range(N):
        pos[i] += 0.5 * dt * vel[i]
    
    scatter.set_offsets(pos)
    f.value =100*step/500
    return [scatter]

from IPython.display import HTML
import numpy as np
from matplotlib import animation
import matplotlib.pyplot as plt

fig = plt.gcf()
ax = plt.gca()
ax.set_xlim(-box_length, +box_length)
ax.set_ylim(-box_length, +box_length)
scatter = ax.scatter(pos[:,0], pos[:,1], s=250)
print("Running calclations, please wait!")
anim = animation.FuncAnimation(fig, timestep, np.arange(1, 500), interval=10, blit=True)
HTML(anim.to_html5_video())

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


Running calclations, please wait!
